# Text Classification using RNN

## Business Problem

Spam and ham classification plays a crucial role in contemporary communication systems, especially in the context of electronic communication channels like emails and text messages. With the exponential growth of digital communication, the significance of accurate spam and ham classification has become increasingly important for several reasons:

1. **User Experience:** Efficient spam filtering ensures a better user experience by preventing unwanted and irrelevant content from reaching users' inboxes. Users are less likely to be inundated with unsolicited messages, leading to a cleaner and more organized communication environment.

2. **Productivity:** Spam filtering contributes to increased productivity as users spend less time sifting through irrelevant or potentially harmful messages. It allows individuals and organizations to focus on legitimate and valuable communication, improving overall workflow efficiency.

3. **Security:** Many spam messages are associated with phishing attempts, scams, and malware distribution. Effective spam filtering acts as a frontline defense against cyber threats by identifying and isolating malicious content, thereby enhancing the security posture of individuals and organizations.

4. **Resource Optimization:** By reducing the volume of spam that enters email servers and messaging platforms, resources such as storage, bandwidth, and processing power can be optimized. This is particularly crucial for large-scale email providers and enterprises dealing with vast amounts of communication data.

5. **Brand Reputation:** For businesses and organizations, spam filtering is essential for maintaining a positive brand reputation. Preventing spam from reaching customers' inboxes ensures that legitimate messages are not overlooked or associated with undesirable content, preserving trust and credibility.

6. **Regulatory Compliance:** Compliance with data protection and privacy regulations often requires organizations to implement measures to protect users from unwanted communication. Adequate spam and ham classification helps in meeting regulatory requirements and avoiding potential legal implications.

## Data  
**Context**  
The SMS Spam Collection is a set of SMS tagged messages that have been collected for SMS Spam research. It contains one set of SMS messages in English of 5,572 messages, tagged acording being ham (legitimate) or spam.  

**Acknowledgement**  
The original dataset can be found [here](https://archive.ics.uci.edu/dataset/228/sms+spam+collection)

# Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
import os
from nltk.corpus import stopwords
from nltk import *
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import pickle
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import nltk
from sklearn.preprocessing import LabelEncoder
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rushi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
sp=pd.read_csv("C:\Dataset\spam1.csv" ,encoding ='latin-1')

# Preprocessing the Data

In [3]:
# reading the file
file_content = pd.read_csv("C:\Dataset\spam1.csv" ,encoding ='latin-1')
file_content

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
6771,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
6772,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
6773,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
6774,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
# checking only the email column
file_content["v2"]

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
6771    This is the 2nd time we have tried 2 contact u...
6772                Will Ì_ b going to esplanade fr home?
6773    Pity, * was in mood for that. So...any other s...
6774    The guy did some bitching but I acted like i'd...
6775                           Rofl. Its true to its name
Name: v2, Length: 6776, dtype: object

In [6]:
# removing the stop words
stop = stopwords.words("english")
file_content["v2"] = file_content["v2"].apply(
    lambda x: " ".join(x for x in x.split() if x not in stop))

# delete unwanted columns
Email_Data = file_content[['v1', 'v2']]

#rename column names
Email_Data = Email_Data.rename(columns={"v1":"Target", "v2":"Email"})
Email_Data


,Target,Email
0,ham,"Go jurong point, crazy.. Available bugis n gre..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry 2 wkly comp win FA Cup final tkts 2...
3,ham,U dun say early hor... U c already say...
4,ham,"Nah I think goes usf, lives around though"
...,...,...
6771,spam,This 2nd time tried 2 contact u. U å£750 Pound...
6772,ham,Will Ì_ b going esplanade fr home?
6773,ham,"Pity, * mood that. So...any suggestions?"
6774,ham,The guy bitching I acted like i'd interested b...


In [7]:
Email_Data.Target.value_counts()

Target
ham     5854
spam     922
Name: count, dtype: int64

In [8]:
# Delete punctuations, convert text to lowercase, and remove double spaces

Email_Data['Email'] = Email_Data['Email'].apply(
    lambda x: re.sub('[!@#$:).;,?&]', '', x.lower()))
Email_Data['Email'] = Email_Data['Email'].apply(
    lambda x: re.sub(' +', ' ', x))
Email_Data['Email'].head(5)


0    go jurong point crazy available bugis n great ...
1                              ok lar joking wif u oni
2    free entry 2 wkly comp win fa cup final tkts 2...
3                  u dun say early hor u c already say
4             nah i think goes usf lives around though
Name: Email, dtype: object

In [9]:
# Separating text(input) and target classes
list_sentences_rawdata = Email_Data["Email"].fillna("_na_").values
list_classes = ["Target"]
target = Email_Data[list_classes].values
To_Process = Email_Data[["Email", "Target"]]

In [10]:
target

array([['ham'],
       ['ham'],
       ['spam'],
       ...,
       ['ham'],
       ['ham'],
       ['ham']], dtype=object)

In [11]:
To_Process

,Email,Target
0,go jurong point crazy available bugis n great ...,ham
1,ok lar joking wif u oni,ham
2,free entry 2 wkly comp win fa cup final tkts 2...,spam
3,u dun say early hor u c already say,ham
4,nah i think goes usf lives around though,ham
...,...,...
6771,this 2nd time tried 2 contact u u å£750 pound ...,spam
6772,will ì_ b going esplanade fr home,ham
6773,pity * mood that soany suggestions,ham
6774,the guy bitching i acted like i'd interested b...,ham


In [12]:
# Preparing data for model building
train, test = train_test_split(To_Process, test_size=0.3)

# Defining the sequence lengths, max number of words and embedding dimensions
# sequence length of each sentence. If more, truncate. If less, pad with zeros
MAX_SEQUENCE_LENGTH = 300

# Top 20000 frequently occuring words
MAX_NB_WORDS = 20000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train.Email)
train_sequences = tokenizer.texts_to_sequences(train.Email)
test_sequences = tokenizer.texts_to_sequences(test.Email)

# dictionary containing words and their index
word_index = tokenizer.word_index

# total words in the corpus
print("Found %s unique tokens." % len(word_index))
c
# get only the top frequent words on train
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# get only the top frequent word on text
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(train_data.shape)
print(test_data.shape)

Found 7886 unique tokens.
(4743, 300)
(2033, 300)


In [13]:
train_labels = train["Target"]
test_labels = test["Target"]

# Convert the character array to numeric array. Assigns levels to unique labels
le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)
print(le.classes_)
print(np.unique(train_labels, return_counts=True))
print(np.unique(test_labels, return_counts=True))

['ham' 'spam']
(array([0, 1]), array([4080,  663], dtype=int64))
(array([0, 1]), array([1774,  259], dtype=int64))


In [14]:
# changing data types
labels_train = to_categorical(np.asarray(train_labels))
labels_test = to_categorical(np.asarray(test_labels))
print('Shape of data tensor:', train_data.shape)
print('Shape of label tensor:', labels_train.shape)
print('Shape of label tensor:', labels_test.shape)


Shape of data tensor: (4743, 300)
Shape of label tensor: (4743, 2)
Shape of label tensor: (2033, 2)


In [15]:
EMBEDDING_DIM = 100
print(MAX_SEQUENCE_LENGTH)

300


# Building the Model

In [16]:
# model building
print("Training Simple RNN")
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(SimpleRNN(2))
model.add(Dense(2, activation="softmax"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Training Simple RNN


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
model.fit(train_data, labels_train, batch_size=16, epochs=5, validation_data=(test_data, labels_test))

Epoch 1/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 42s 119ms/step - accuracy: 0.8370 - loss: 0.5784 - val_accuracy: 0.9326 - val_loss: 0.3989
Epoch 2/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 33s 110ms/step - accuracy: 0.9596 - loss: 0.3061 - val_accuracy: 0.9174 - val_loss: 0.3137
Epoch 3/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 32s 108ms/step - accuracy: 0.9852 - loss: 0.1870 - val_accuracy: 0.9188 - val_loss: 0.2746
Epoch 4/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 33s 112ms/step - accuracy: 0.9937 - loss: 0.1217 - val_accuracy: 0.9090 - val_loss: 0.2563
Epoch 5/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 34s 113ms/step - accuracy: 0.9953 - loss: 0.0856 - val_accuracy: 0.9061 - val_loss: 0.2446


# Model Evaluation

In [18]:
# Define class names
class_names = ['ham', 'spam']

# Predictions
predicted_Srnn = model.predict(test_data)

# Converting probabilities to binary predictions
binary_predictions = np.round(predicted_Srnn)

# Calculating precision, recall, fscore, and support
precision, recall, fscore, support = precision_recall_fscore_support(
                              labels_test,
                              binary_predictions,
                              average='weighted')

print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F-score: {}'.format(fscore))
print('Support: {}'.format(support))

print("############################")

# Classification Report with class names
report = classification_report(labels_test, binary_predictions, target_names=class_names)
print(report)


64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step
Precision: 0.9091220879792187
Recall: 0.9060501721593703
F-score: 0.9074593157156039
Support: None
############################
              precision    recall  f1-score   support

         ham       0.95      0.94      0.95      1774
        spam       0.62      0.67      0.64       259

   micro avg       0.91      0.91      0.91      2033
   macro avg       0.79      0.80      0.80      2033
weighted avg       0.91      0.91      0.91      2033
 samples avg       0.91      0.91      0.91      2033



# Deploying the Model on Gradio

In [27]:
model.save('model.keras')


In [28]:
import pickle
# Save the tokenizer to a file
with open('tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)


In [29]:
# !pip install gradio

In [19]:
import gradio as gr

In [20]:
# Loading the trained model
model = load_model('model.keras')
# Loading the tokenizer from the file
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

def preprocess_input_sequence(input_sequence):
    # preprocessing the input text.
    stop = stopwords.words("english")
    input_sequence = " ".join(x for x in input_sequence.split() if x not in stop)
    input_sequence = re.sub('[!@#$:).;,?&]', '', input_sequence.lower())
    input_sequence = re.sub(' +', ' ', input_sequence)
    tokenized_sequence = tokenizer.texts_to_sequences([input_sequence])
    processed_input_sequence = pad_sequences(tokenized_sequence,
                                    maxlen=300)[0]
    return processed_input_sequence

def predict_sequence(input_sequence):
    # Preprocess the input_sequence
    processed_input = preprocess_input_sequence(input_sequence)

    # Model prediction
    prediction = model.predict(np.array([processed_input]))

    # Converting probability to class (assuming binary classification)
    predicted_class = int(np.round(prediction.flatten()[0]))

    # Mapping class to 'ham' or 'spam'
    result = 'ham' if predicted_class == 0 else 'spam'

    return result

iface = gr.Interface(
    fn=predict_sequence,
    inputs="text",
    outputs="text",  # Output as text for displaying 'ham' or 'spam'
    live=True
)

iface.launch()


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━

# End of the Project